# 借助 pyzx 库进行线路化简

In [1]:
from opt_qcir_pyzx import *
from pyzx.graph.graph_s import GraphS

qcis = load_qcis_example(0)
info = qcis_info(qcis)
print(info)
qcis = render_qcis(qcis, {k: 1 for k in info.param_names})

qtape = qcis_to_qtape(qcis)
print('>> qtape length before:', len(qtape))

CircuitInfo(n_qubits=4, n_gates=284, n_gate_types=5, n_depth=64, qubit_ids=[0, 1, 2, 3], gate_types=['CZ', 'H', 'RZ', 'X2M', 'X2P'], param_names=['d1_0', 's_0'])
>> qtape length before: 285


In [2]:
g: GraphS = to_zx(qtape)
print(g)
c = zx.Circuit.from_graph(g)
print(c)

g_vis = g.copy()
g_vis.normalize()
zx.draw(g_vis)

Graph(356 vertices, 416 edges)
Circuit(4 qubits, 0 bits, 284 gates)


⚪ Full Reduce

In [3]:
print('[full_reduce]')

g_fr = g.copy()
zx.full_reduce(g_fr, quiet=False)
print()
print(g_fr)
c_fr = zx.extract_circuit(g_fr.copy())
print(c_fr)

assert zx.compare_tensors(c, c_fr)
assert c.verify_equality(c_fr)

g_vis_fr = g_fr.copy()
g_vis_fr.normalize()
zx.draw(g_vis_fr)

[full_reduce]
spider_simp: 63. 45. 24. 8.  4 iterations
id_simp: 52. 20.  2 iterations
spider_simp: 29. 6. 3. 3. 1.  5 iterations
pivot_simp: 12. 6. 3. 2. 1.  5 iterations
lcomp_simp: 9. 5. 2.  3 iterations
id_simp: 1.  1 iterations
spider_simp: 1.  1 iterations
pivot_simp: 1. 1.  2 iterations
pivot_gadget_simp: 4. 2.  2 iterations
gadget_simp: 3.  1 iterations

Graph(34 vertices, 77 edges)
Circuit(4 qubits, 0 bits, 86 gates)


In [4]:
qtape_opt = from_zx(c_fr.to_graph())
print('>> qtape length after:', len(qtape_opt))
qcis = qtape_to_qcis(qtape_opt)

>> qtape length after: 92


⚪ Teleport Reduce

In [5]:
print('[teleport_reduce]')

g_tr = g.copy()
zx.teleport_reduce(g_tr, quiet=False)
print()
print(g_tr)
c_tr = zx.Circuit.from_graph(g_tr.copy())
print(c_tr)

assert zx.compare_tensors(c, c_tr)
assert c.verify_equality(c_tr)

g_vis_tr = g_tr.copy()
g_vis_tr.normalize()
zx.draw(g_vis_tr)

[teleport_reduce]
spider_simp: 63. 45. 24. 8.  4 iterations
id_simp: 52. 20.  2 iterations
spider_simp: 29. 6. 3. 3. 1.  5 iterations
pivot_simp: 12. 6. 3. 2. 1.  5 iterations
lcomp_simp: 9. 5. 2.  3 iterations
id_simp: 1.  1 iterations
spider_simp: 1.  1 iterations
pivot_simp: 1. 1.  2 iterations
pivot_gadget_simp: 4. 2.  2 iterations
gadget_simp: 3.  1 iterations

Graph(356 vertices, 416 edges)
Circuit(4 qubits, 0 bits, 284 gates)


In [6]:
qtape_opt = from_zx(g_tr)
print('>> qtape length after:', len(qtape_opt))
qcis = qtape_to_qcis(qtape_opt)

>> qtape length after: 284
